In [ ]:
import uproot
import numpy as np
import glob
import os
import sys
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_curve, auc, accuracy_score
import tensorflow.keras.backend as K
from sklearn.preprocessing import StandardScaler, LabelEncoder
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import *
from tensorflow import keras
from tensorflow.keras import layers, models, Model
import h5py

from utils import *

np.random.seed(42)
tf.random.set_seed(42)

In [ ]:
n_per_sample = 10000

with h5py.File('HHbbbb.h5', 'r') as f:
    X_HHbbbb_isHS = tf.cast(f['HS'][:n_per_sample], tf.float32)
    X_HHbbbb_isPU = tf.cast(f['PU'][:n_per_sample], tf.float32)

with h5py.File('PJZ0.h5', 'r') as f:
    X_PJZ0 = tf.cast(f['data'][:n_per_sample], tf.float32)

print(X_HHbbbb_isHS.shape)
print(X_HHbbbb_isPU.shape)
print(X_PJZ0.shape)

In [ ]:
plot_layers(event_idx=1, X=X_HHbbbb_isHS+X_HHbbbb_isPU, label='[HHbbbb, PU=200]')
plot_layers(event_idx=1, X=X_HHbbbb_isHS, label='[HHbbbb, PU=0]')
plot_layers(event_idx=2, X=X_PJZ0, label='[QCD dijet, PU=200]')

In [ ]:
target_pu = 100
x_augmented = augment_pu(image=X_HHbbbb_isPU[0], target_pu=target_pu, shift_phi=True)

plot_layers(event_idx=None, X=X_HHbbbb_isPU[0], label='[Pure PU, 200]')
plot_layers(event_idx=None, X=x_augmented, label=f'[Pure PU, aug. {target_pu}]')

## build models

In [ ]:
input_shape = (64, 50, 6)
embedding_dim = 128
projection_dim = 64
c_inv = 25
c_var = 25
c_cov = 1

def build_encoder(input_shape=input_shape, embedding_dim=embedding_dim):
    inputs = tf.keras.Input(shape=input_shape)
    x = tf.keras.layers.Conv2D(64, (3, 3), activation='relu', padding='same')(inputs)
    x = tf.keras.layers.MaxPooling2D((2, 2))(x)
    x = tf.keras.layers.Conv2D(128, (3, 3), activation='relu', padding='same')(x)
    x = tf.keras.layers.GlobalAveragePooling2D()(x)
    x = tf.keras.layers.Dense(embedding_dim)(x)
    outputs = tf.keras.layers.LayerNormalization()(x)

    return tf.keras.Model(inputs, outputs, name="encoder")


def build_projection_head(embedding_dim=embedding_dim, projection_dim=projection_dim):
    inputs = tf.keras.Input(shape=(embedding_dim,))
    x = tf.keras.layers.Dense(embedding_dim, activation='relu')(inputs)
    outputs = tf.keras.layers.Dense(projection_dim)(x)

    return tf.keras.Model(inputs, outputs, name="projection_head")


def vicreg_loss(z1, z2, c_inv=c_inv, c_var=c_var, c_cov=c_cov, epsilon=0.0001):
    # invariance between positive views
    loss_inv = tf.reduce_mean(tf.square(z1 - z2))

    # maximize variance per feature dim across sample batch (avoid collapse--learning constant vector)
    std_z1 = tf.sqrt(tf.math.reduce_variance(z1, axis=0) + epsilon)
    std_z2 = tf.sqrt(tf.math.reduce_variance(z2, axis=0) + epsilon)
    loss_var = tf.reduce_mean(tf.nn.relu(1 - std_z1)) + tf.reduce_mean(tf.nn.relu(1 - std_z2))

    # minimize covariance between feature dims (to reduce learning feature redundancy)
    z1_centered = z1 - tf.reduce_mean(z1, axis=0)
    z2_centered = z2 - tf.reduce_mean(z2, axis=0)
    batch_size = tf.cast(tf.shape(z1)[0], tf.float32)
    # covariance matrices for z1 z2
    cov_z1 = tf.matmul(tf.transpose(z1_centered), z1_centered) / (batch_size - 1)
    cov_z2 = tf.matmul(tf.transpose(z2_centered), z2_centered) / (batch_size - 1)
    # get diagonal parts
    diag_z1 = tf.linalg.diag(tf.linalg.diag_part(cov_z1))
    diag_z2 = tf.linalg.diag(tf.linalg.diag_part(cov_z2))
    # subtract diagonal parts from cov matrices to get off-diagonal parts (cov between features)
    loss_cov_z1 = tf.reduce_sum(tf.square(cov_z1 - diag_z1)) / tf.cast(tf.shape(z1)[1], tf.float32)
    loss_cov_z2 = tf.reduce_sum(tf.square(cov_z2 - diag_z2)) / tf.cast(tf.shape(z2)[1], tf.float32)
    loss_cov = loss_cov_z1 + loss_cov_z2

    loss_inv *= c_inv
    loss_var *= c_var
    loss_cov *= c_cov

    loss = loss_inv + loss_var + loss_cov

    return loss, loss_inv, loss_var, loss_cov


class VICRegModel(tf.keras.Model):
    def __init__(self, encoder, projection_head, c_inv=c_inv, c_var=c_var, c_cov=c_cov, **kwargs):
        super(VICRegModel, self).__init__(**kwargs)
        self.encoder = encoder
        self.projection_head = projection_head
        self.c_inv = c_inv
        self.c_var = c_var
        self.c_cov = c_cov

    def compile(self, optimizer, **kwargs):
        super(VICRegModel, self).compile(**kwargs)
        self.optimizer = optimizer

    def train_step(self, data):
        # view1, view2
        x1, x2 = data
        with tf.GradientTape() as tape:
            emb1 = self.encoder(x1, training=True)
            emb2 = self.encoder(x2, training=True)

            z1 = self.projection_head(emb1, training=True)
            z2 = self.projection_head(emb2, training=True)

            loss, loss_inv, loss_var, loss_cov = vicreg_loss(z1, z2,
                                                             c_inv=self.c_inv,
                                                             c_var=self.c_var,
                                                             c_cov=self.c_cov)
        
        vars = self.encoder.trainable_variables + self.projection_head.trainable_variables
        grads = tape.gradient(loss, vars)
        self.optimizer.apply_gradients(zip(grads, vars))

        return {"loss": loss, "loss_inv": loss_inv, "loss_var": loss_var, "loss_cov": loss_cov}


def build_classifier(encoder, input_shape=input_shape):
    # update or freeze the encoder weights
    encoder.trainable = False

    inputs = tf.keras.Input(shape=input_shape)
    # option-training here concerns about training/inference mode in dropout, batchnorm etc. 
    x = encoder(inputs, training=False)
    x = tf.keras.layers.Dense(64, activation='relu')(x)
    outputs = tf.keras.layers.Dense(1, activation='sigmoid')(x)

    return tf.keras.Model(inputs, outputs, name="classifier")

## train vicreg

In [ ]:
batch_size = 1024
pu_min = 100
pu_max = 200
steps_per_epoch = (X_HHbbbb_isHS.shape[0] + X_PJZ0.shape[0]) // batch_size

gen_data_contrastive = generate_batch_for_contrastive(X_hs=X_HHbbbb_isHS,
                                                      X_pu=X_HHbbbb_isPU,
                                                      X_bkg=X_PJZ0,
                                                      pu_min=pu_min,
                                                      pu_max=pu_max,
                                                      batch_size=batch_size)

encoder = build_encoder(input_shape=input_shape, embedding_dim=embedding_dim)
projection_head = build_projection_head(embedding_dim=embedding_dim, projection_dim=projection_dim)

vicreg_model = VICRegModel(encoder=encoder, projection_head=projection_head, c_inv=c_inv, c_var=c_var, c_cov=c_cov)
vicreg_model.compile(optimizer=tf.keras.optimizers.legacy.Adam(learning_rate=0.0005))

encoder.summary()
print('\n\n')
projection_head.summary()

history = vicreg_model.fit(gen_data_contrastive, steps_per_epoch=steps_per_epoch, epochs=40)

plt.figure(figsize = (6,4))
axes = plt.subplot(1,1,1)
axes.plot(history.history['loss'], label = 'loss (total)')
axes.plot(history.history['loss_inv'], label = 'invariance')
axes.plot(history.history['loss_var'], label = 'variance')
axes.plot(history.history['loss_cov'], label = 'covariance')
axes.legend(loc = "upper right")
axes.set_xlabel('Epoch')
axes.set_ylabel('Loss')

In [ ]:
if os.path.exists('weights_encoder.h5'):
    os.remove('weights_encoder.h5')
    
if os.path.exists('weights_projection_head.h5'):
    os.remove('weights_projection_head.h5')

encoder.save_weights('weights_encoder.h5')
projection_head.save_weights('weights_projection_head.h5')

In [ ]:
encoder_loaded = build_encoder(input_shape=input_shape, embedding_dim=embedding_dim)
encoder_loaded.load_weights('weights_encoder.h5')

projection_head_loaded = build_projection_head(embedding_dim=embedding_dim, projection_dim=projection_dim)
projection_head_loaded.load_weights('weights_projection_head.h5')

vicreg_model_loaded = VICRegModel(encoder=encoder_loaded,
                                  projection_head=projection_head_loaded,
                                  c_inv=c_inv,
                                  c_var=c_var,
                                  c_cov=c_cov)
vicreg_model_loaded.compile(optimizer=tf.keras.optimizers.legacy.Adam(learning_rate=0.0005))

## train classifier

In [ ]:
batch_size = 1024
pu_min = 100
pu_max = 200
steps_per_epoch = (X_HHbbbb_isHS.shape[0] + X_PJZ0.shape[0]) // batch_size

gen_data_classification = generate_batch_for_classifier(X_hs=X_HHbbbb_isHS,
                                                        X_pu=X_HHbbbb_isPU,
                                                        X_bkg=X_PJZ0,
                                                        pu_min=pu_min,
                                                        pu_max=pu_max,
                                                        batch_size=batch_size)

classifier = build_classifier(encoder=encoder, input_shape=input_shape)
classifier.compile(optimizer=tf.keras.optimizers.legacy.Adam(learning_rate=0.0005),
                   loss='binary_crossentropy',
                   metrics=['accuracy'])

history = classifier.fit(gen_data_classification, steps_per_epoch=steps_per_epoch, epochs=20)

plt.figure(figsize = (6,4))
axes = plt.subplot(1,1,1)
axes.plot(history.history['loss'], label = 'loss')
axes.plot(history.history['accuracy'], label = 'accuracy')
axes.legend(loc = "upper right")
axes.set_xlabel('Epoch')
axes.set_ylabel('Loss')

In [ ]:
if os.path.exists('weights_classifier.h5'):
    os.remove('weights_classifier.h5')

classifier.save_weights('weights_classifier.h5')

In [ ]:
encoder_loaded = build_encoder(input_shape=input_shape, embedding_dim=embedding_dim)
encoder_loaded.load_weights('weights_encoder.h5')

classifier_loaded = build_classifier(encoder=encoder_loaded, input_shape=input_shape)
classifier_loaded.load_weights('weights_classifier.h5')
classifier.compile(optimizer=tf.keras.optimizers.legacy.Adam(learning_rate=0.0005),
                   loss='binary_crossentropy',
                   metrics=['accuracy'])

## evaluate